In [97]:
# coding: utf-8
# The Midterm exam
# target = 'poi' 嫌疑人的標記

import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import datasets, metrics

In [71]:
# 設定 data_path
dir_data = 'D:/data/3rd-ml100marathon-midterm/'

# import data
df_train = pd.read_csv(dir_data + 'train_data.csv')
df_test = pd.read_csv(dir_data + 'test_features.csv')
ids =  df_test['name']
# 將poi提取出來
# poi的形式為numpy.bool_形式，雖顯示TURE/FALSE，但可直接用1/0做為條件盼動，而不用另外轉換為數值型態
train_Y = (df_train['poi'].astype('float')).astype('int')
df_train = df_train.drop(['name', 'email_address', 'poi'] , axis=1)
df_test = df_test.drop(['name', 'email_address'] , axis=1)

In [55]:
#各變項list
financial_lst = ['salary', 'deferral_payments', 'total_payments', 'loan_advances', 'bonus', 'restricted_stock_deferred', 'deferred_income', 'total_stock_value', 'expenses', 'exercised_stock_options', 'other', 'long_term_incentive', 'restricted_stock', 'director_fees']
email_lst = ['to_messages', 'email_address', 'from_poi_to_this_person', 'from_messages', 'from_this_person_to_poi', 'shared_receipt_with_poi']

In [72]:
df_train.head(5)

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1750000.0,NaN,-3504386.0,NaN,19794175.0,46950.0,18.0,42.0,4.0,NaN,1617011.0,174839.0,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,5600000.0,NaN,NaN,NaN,19250000.0,29336.0,108.0,88.0,30.0,NaN,1920000.0,22122.0,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,200000.0,NaN,-4167.0,NaN,1624396.0,22884.0,39.0,13.0,14.0,NaN,NaN,1573324.0,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,800000.0,NaN,NaN,NaN,NaN,118134.0,NaN,NaN,NaN,NaN,602671.0,907502.0,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,1250000.0,NaN,-262500.0,NaN,NaN,35818.0,144.0,199.0,25.0,NaN,375304.0,486.0,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0


In [61]:
# check variable info
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 19 columns):
bonus                        61 non-null float64
deferral_payments            28 non-null float64
deferred_income              34 non-null float64
director_fees                13 non-null float64
exercised_stock_options      81 non-null float64
expenses                     73 non-null float64
from_messages                65 non-null float64
from_poi_to_this_person      65 non-null float64
from_this_person_to_poi      65 non-null float64
loan_advances                2 non-null float64
long_term_incentive          49 non-null float64
other                        69 non-null float64
restricted_stock             82 non-null float64
restricted_stock_deferred    10 non-null float64
salary                       73 non-null float64
shared_receipt_with_poi      65 non-null float64
to_messages                  65 non-null float64
total_payments               96 non-null float64
total_stock_va

In [60]:
# check 資料的描述性統計
df_train.describe()

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
count,6.100000e+01,2.800000e+01,3.400000e+01,13.000000,8.100000e+01,73.000000,65.000000,65.000000,65.000000,2.000000e+00,4.900000e+01,6.900000e+01,8.200000e+01,10.000000,7.300000e+01,65.000000,65.000000,9.600000e+01,9.800000e+01
mean,1.147436e+06,6.344374e+05,-4.625664e+05,89397.846154,2.985081e+06,51040.547945,711.323077,64.800000,40.092308,4.096250e+07,7.926171e+05,4.471774e+05,1.294855e+06,-221885.700000,2.739025e+05,1111.369231,2156.061538,2.590977e+06,3.527136e+06
std,1.505189e+06,8.603646e+05,8.095392e+05,41143.391399,6.004174e+06,47596.682104,2074.497628,91.863214,88.901407,5.736404e+07,9.504645e+05,1.341564e+06,2.498335e+06,205191.374121,1.716647e+05,1165.852016,2811.676718,1.056645e+07,7.182997e+06
min,7.000000e+04,-1.025000e+05,-3.504386e+06,3285.000000,3.285000e+03,148.000000,12.000000,0.000000,0.000000,4.000000e+05,7.102300e+04,2.000000e+00,4.409300e+04,-560222.000000,4.770000e+02,2.000000,57.000000,1.480000e+02,-4.409300e+04
25%,4.500000e+05,7.656750e+04,-5.527032e+05,101250.000000,4.004780e+05,18834.000000,19.000000,10.000000,0.000000,2.068125e+07,2.750000e+05,9.720000e+02,2.689220e+05,-389621.750000,2.061210e+05,178.000000,517.000000,3.024025e+05,4.211518e+05
50%,7.500000e+05,1.951900e+05,-1.175340e+05,108579.000000,8.500100e+05,41953.000000,45.000000,28.000000,7.000000,4.096250e+07,4.221580e+05,5.238200e+04,4.628225e+05,-139856.500000,2.516540e+05,599.000000,1088.000000,1.106740e+06,9.979710e+05
75%,1.000000e+06,8.342052e+05,-2.708325e+04,112492.000000,2.165172e+06,59175.000000,215.000000,88.000000,27.000000,6.124375e+07,8.318090e+05,3.620960e+05,9.664905e+05,-77953.250000,2.885890e+05,1902.000000,2649.000000,1.985668e+06,2.493616e+06
max,8.000000e+06,2.964506e+06,-1.042000e+03,125034.000000,3.434838e+07,228763.000000,14368.000000,528.000000,411.000000,8.152500e+07,5.145434e+06,1.035973e+07,1.476169e+07,44093.000000,1.111258e+06,4527.000000,15149.000000,1.035598e+08,4.911008e+07


In [73]:
# 確認缺失值的狀態
def na_check(df_data):
    data_na = (df_data.isnull().sum() / len(df_data)) * 100
    data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)
    missing_data = pd.DataFrame({'Missing Ratio' :data_na})
    display(missing_data)
na_check(df_train)

,Missing Ratio
loan_advances,98.230088
restricted_stock_deferred,91.150442
director_fees,88.495575
deferral_payments,75.221239
deferred_income,69.911504
long_term_incentive,56.637168
bonus,46.017699
from_this_person_to_poi,42.477876
from_poi_to_this_person,42.477876
from_messages,42.477876


In [79]:
na_check(df_test)

,Missing Ratio
loan_advances,93.939394
director_fees,87.878788
restricted_stock_deferred,75.757576
deferral_payments,66.666667
deferred_income,54.545455
long_term_incentive,48.484848
from_this_person_to_poi,36.363636
exercised_stock_options,36.363636
from_messages,36.363636
from_poi_to_this_person,36.363636


In [74]:
#找出財務變項中值=0的個數
for var in financial_lst:
    _df = df_train[df_train[var] == 0]
    print(f'{var} 等於0的個數為:', len(_df))

salary 等於0的個數為: 0
deferral_payments 等於0的個數為: 0
total_payments 等於0的個數為: 0
loan_advances 等於0的個數為: 0
bonus 等於0的個數為: 0
restricted_stock_deferred 等於0的個數為: 0
deferred_income 等於0的個數為: 0
total_stock_value 等於0的個數為: 0
expenses 等於0的個數為: 0
exercised_stock_options 等於0的個數為: 0
other 等於0的個數為: 0
long_term_incentive 等於0的個數為: 0
restricted_stock 等於0的個數為: 0
director_fees 等於0的個數為: 0


* 在上述財務變項中，有0的紀錄均為0，再根據財務報表的紀錄狀況可判定，財務變項若沒有記錄代表沒有該項行為之事實
* 故在資料處理上標記為0，顯示該名人員沒有此項行為。
* 同樣，在email相關的變項中，若是NAN代表沒有該項行為之事實，一樣轉為0

In [80]:
#將缺失值紀錄轉為0
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

In [76]:
# 再確認一次缺失裝況
na_check(df_train)

,Missing Ratio


In [86]:
#使用logistic regression看預測情況
# create model
log_reg = LogisticRegression(C = 0.005, max_iter = 200, random_state=5)

# training model
log_reg.fit(df_train, train_Y)
scores = cross_val_score(log_reg, df_train, train_Y, cv=5).mean()
print(scores)

c:\python36\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
c:\python36\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_

0.7786561264822133


c:\python36\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [111]:
# 請注意羅吉斯迴歸是分類預測 (會輸出 0 的機率, 與 1 的機率), 而我們只需要留下 1 的機率這排
log_reg_pred = log_reg.predict_proba(df_test)[:, 1]
score = cross_val_score(log_reg, df_train, train_Y, cv=5).mean()
print('Logistic Regression score:',score)

c:\python36\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
c:\python36\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_

Logistic Regression score: 0.7786561264822133


c:\python36\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


* 分數僅有0.778

In [93]:
#嘗試用其他模型
# 使用三種模型 : 邏輯斯迴歸 / 梯度提升機 / 隨機森林, 參數使用 Random Search 尋找
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier


In [106]:
gdbt = GradientBoostingClassifier(tol=100, subsample=0.75, n_estimators=250, max_features=0.5,
                                  max_depth=6, learning_rate=0.03)
rf = RandomForestClassifier(n_estimators=100, min_samples_split=2, min_samples_leaf=1, 
                            max_features='sqrt', max_depth=6, bootstrap=True)

In [112]:
# 梯度提升機預測檔 
gdbt.fit(df_train, train_Y)
gdbt_pred = gdbt.predict_proba(df_test)[:,1]
score2 = cross_val_score(gdbt, df_train, train_Y, cv=5).mean()
print('Gradient Boosting score:', score2)

Gradient Boosting score: 0.8936758893280633


* 若用梯度提升機，則分數可提高到0.893

In [113]:
# 使用隨機森林 Ramdom Forest
rf.fit(df_train, train_Y)
rf_pred = rf.predict_proba(df_test)[:,1]
score3 = cross_val_score(rf, df_train, train_Y, cv=5).mean()
print('Random Forest score:',score3)

Random Forest score: 0.8936758893280633


* 使用隨機森林的分數為:0.893

In [117]:
# 以梯度提升機的分數作提交
submission = pd.DataFrame({'Name': ids, 'poi': gdbt_pred})
submission.to_csv('Day_051_053_midterm_exam_Enron_Fraud_by_GDBT.csv', index=False)

In [116]:
submission

,Name,poi
0,BELDEN TIMOTHY N,0.039808
1,BOWEN JR RAYMOND M,0.049772
2,HANNON KEVIN P,0.256830
3,DELAINEY DAVID W,0.000135
4,CAUSEY RICHARD A,0.105530
5,HICKERSON GARY J,0.091665
6,FREVERT MARK A,0.914184
7,CHAN RONNIE,0.000187
8,DONAHUE JR JEFFREY M,0.000609
9,REYNOLDS LAWRENCE,0.000200


In [121]:
#將財務變項全部轉為類別型
for i in df_train:
    df_train[i] = df_train[i].apply(lambda x: 0 if x==0 else 1)

In [122]:
# 梯度提升機預測檔 
gdbt.fit(df_train, train_Y)
gdbt_pred = gdbt.predict_proba(df_test)[:,1]
score2 = cross_val_score(gdbt, df_train, train_Y, cv=5).mean()
print('Gradient Boosting score:', score2)

Gradient Boosting score: 0.859288537549407


In [124]:
# 以梯度提升機的分數作提交
submission = pd.DataFrame({'Name': ids, 'poi': gdbt_pred})
submission.to_csv('Day_051_053_midterm_exam_Enron_Fraud_by_GDBT2.csv', index=False)